In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
%matplotlib inline

from astrobee_strips import STRIPS, Node, Queue, get_plan

from sys import path
path.append('/home/acauligi/Software')

import pdb
import numpy as np
from casadi import *

# STRIPS planner

In [2]:
strip = STRIPS()
n0 = Node(strip)
qq = Queue()
qq.queue(n0)

operators = ['dock_objA_dockA', 'dock_objA_dockB', 'dock_objA_dockC', \
            'dock_objB_dockA', 'dock_objB_dockB', 'dock_objB_dockC', \
            'undock_objA_dockA', 'undock_objA_dockB', 'undock_objA_dockC', \
            'undock_objB_dockA', 'undock_objB_dockB', 'undock_objB_dockC', \
            'grasp_objA', 'grasp_objB']

max_depth = 5

In [3]:
while not qq.is_empty():
    if all([node.depth >= max_depth for node in qq.list]):
        print('Max depth violated')
        break

    leaf_node = qq.dequeue()
    for operator in operators:
        new_leaf = leaf_node.act(operator)
        if all(new_leaf.strip.state == leaf_node.strip.state):
            continue
        qq.queue(new_leaf)

    if any([node.solved for node in qq.list]):
        print('solved')
        break

solved


In [4]:
solved_node = [node for node in qq.list if node.solved][0]
plan = get_plan(solved_node)
plan

['undock_objA_dockB', 'dock_objA_dockA', 'grasp_objB', 'dock_objB_dockB']

# Ipopt

In [2]:
n = 13
m = 6

N = 50

R = np.eye(m) # cost matrix for 2PBVP

slew_weighting = 1.2 
max_se3_dist = 1.50 

# robot parameters
J = np.array([[0.1083, 0.0, 0.0],
            [0.0, 0.1083, 0.0],
            [0.0, 0.0, 0.1083]])
Jxx, Jyy, Jzz = np.diag(J)
Jinv = np.linalg.inv(J)
mass = 7.0

hard_limit_vel   = 0.50
hard_limit_accel = 0.10
hard_limit_vel   = 5000.
hard_limit_accel = 1000.
hard_limit_omega = 45*np.pi/180
hard_limit_alpha = 50*np.pi/180

# state box constraints
Xlb = np.array([-np.inf,-np.inf,-np.inf, 
-hard_limit_vel/np.sqrt(3),-hard_limit_vel/np.sqrt(3),-hard_limit_vel/np.sqrt(3),
-1.0,-1.0,-1.0,0.0,
-hard_limit_omega/np.sqrt(3),-hard_limit_omega/np.sqrt(3),-hard_limit_omega/np.sqrt(3)])
Xub = np.array([np.inf,np.inf,np.inf, 
hard_limit_vel/np.sqrt(3),hard_limit_vel/np.sqrt(3),hard_limit_vel/np.sqrt(3),
1.0,1.0,1.0,1.0,
hard_limit_omega/np.sqrt(3),hard_limit_omega/np.sqrt(3),hard_limit_omega/np.sqrt(3)])

# control box constraints
Jmin = np.min(np.diag(J))
Ulb = np.array([-mass*hard_limit_accel/np.sqrt(3), -mass*hard_limit_accel/np.sqrt(3), -mass*hard_limit_accel/np.sqrt(3), -Jmin*hard_limit_alpha/np.sqrt(3), -Jmin*hard_limit_alpha/np.sqrt(3), -Jmin*hard_limit_alpha/np.sqrt(3)])
Uub = np.array([mass*hard_limit_accel/np.sqrt(3), mass*hard_limit_accel/np.sqrt(3), mass*hard_limit_accel/np.sqrt(3),  Jmin*hard_limit_alpha/np.sqrt(3), Jmin*hard_limit_alpha/np.sqrt(3), Jmin*hard_limit_alpha/np.sqrt(3)])

# time constraints
taulb = np.array([0.1])
tauub = np.array([100.0])
tau_guess = 50.0 

param_dict =  {'n':n, 'm':m, 'N':N, 'R':R, 'slew_weighting':slew_weighting, 'max_se3_dist':max_se3_dist, 'J':J, 'Jinv':Jinv, 'mass':mass, 
      'hard_limit_vel':hard_limit_vel, 'hard_limit_accel':hard_limit_accel, 'hard_limit_omega':hard_limit_omega, 'hard_limit_alpha':hard_limit_alpha, 
      'Xlb':Xlb, 'Xub':Xub, 'Ulb':Ulb, 'Uub':Uub, 'taulb':taulb, 'tauub':tauub, 'tau_guess':tau_guess}

Declare model variables

In [3]:
rx = MX.sym('rx');    ry = MX.sym('ry');    rz = MX.sym('rz');
qx = MX.sym('qx');    qy = MX.sym('qy');    qz = MX.sym('qz');    qw = MX.sym('qw');
vx = MX.sym('vx');    vy = MX.sym('vy');    vz = MX.sym('vz');
wx = MX.sym('wx');    wy = MX.sym('wy');    wz = MX.sym('wz');
quat = vertcat(qx,qy,qz,qw)
state = vertcat(rx,ry,rz,qx,qy,qz,qw,vx,vy,vz,wx,wy,wz)

Fx = MX.sym('Fx');    Fy = MX.sym('Fy');    Fz = MX.sym('Fz');
Mx = MX.sym('Mx');    My = MX.sym('My');    Mz = MX.sym('Mz');
control = vertcat(Fx,Fy,Fz,Mx,My,Mz)

System dynamics equations

In [4]:
statedot = vertcat(vx,
                vy,
                vz,
                0.5*(qy*wz - qz*wy + qw*wx),
                0.5*(qz*wx - qx*wz + qw*wy),
                0.5*(qx*wy - qy*wx + qw*wz),
                0.5*(-qx*wx - qy*wy - qz*wz),
                1/mass*Fx,
                1/mass*Fy,
                1/mass*Fz,
                (Mx + Jyy*wy*wz - Jzz*wy*wz)/Jxx,
                (My - Jxx*wx*wz + Jzz*wx*wz)/Jyy,
                (Mz + Jxx*wx*wy - Jyy*wx*wy)/Jzz)

f = Function('f', [state,control], [statedot])

Start with an empty NLP

In [5]:
w = []    # the decision variables lbw < w < ubw
lbw = []  # lower bound constraint on dec var
ubw = []  # upper bound constraint on dec var
g = []    # vector for constraints lbg < g(w,p) < ubg
lbg = []  # lower bound on constraints
ubg = []  # upper bound on constraints

Declare decision variables

In [6]:
X = []
for k in range(N):
    Xn = MX.sym('X_'+str(k), n)
    X += [Xn]
    w += [Xn]
    lbw += Xlb.tolist()
    ubw += Xub.tolist()

# control
U = []
for k in range(N-1):
    Un = MX.sym('U_' + str(k), m)
    U += [Un]
    lbw += Ulb.tolist()
    ubw += Uub.tolist()
    w   += [Un]

Discrete update equation

In [7]:
#  ODE right hand side function
f = Function('f', [state,control],[statedot])

# Integrate with Explicit Euler over 0.2 seconds
dh = 0.01  # Time step
xj = state
n_steps = int(10)
for ii in range(n_steps):
    fj = f(xj,control)
    xj += dh*fj

# Discrete time dynamics function
F = Function('F', [state,control],[xj])

Dynamics propagation

In [8]:
X0 = X[0]
Q = X0
for ii in range(N-1):
    Q = F(Q, U[ii])
    g += [Q] 
    lbg += [Xlb.tolist()]
    ubg += [Xub.tolist()]

Quaternion norm constraints

In [9]:
qeps = 1e-1
qnorm = Function('qnorm', [state], [qx**2+qy**2+qz**2+qw**2])
for k in range(1,N):
    Xn = X[k]
    g += [qnorm(Xn)]
    lbg += [1-qeps]
    ubg += [1+qeps]

In [10]:
Xi = np.array([3.,3.,3., 0.,0.,0.,1., 0.05,-0.05,0.02, 0.01,-0.01,0.01])

Xref = DM([0.,0.1,0., 0.5,0.5,0.5,0.5, 0.,0.,0., 0.,0.,0.])

Cost function

In [11]:
J = 0.
for ii in range(N):
    stage_cost = (rx-Xref[0])**2 + (ry-Xref[1])**2 + (rz-Xref[2])**2 + \
        casadi.arccos(casadi.dot(Xref[3:7], quat)) + \
        vx**2 + vy**2 + vz**2 + wx**2 + wy**2 + wz**2 
    stage_cost = casadi.dot(X[ii] - Xref, X[ii] - Xref)
    J += stage_cost
    
for ii in range(N-1):
    J +=  casadi.dot(U[ii], U[ii])

In [14]:
prob = {'f': J, 'x': vertcat(*w), 'g': vertcat(*g)}
solver = nlpsol('S', 'ipopt', prob);

In [12]:
lbw[:n] = Xi.tolist()
ubw[:n] = Xi.tolist()

In [15]:
use_guess = False
if use_guess:
    w0 = [] # initial guess of decision variables
    for ii in range(N):
        w0 += stateguess[ii].tolist()
    for ii in range(N-1):
        w0 += uguess[ii].tolist()
    soln = solver(x0=w0, lbx=lbw, ubx=ubw, lbg=vertcat(*lbg), ubg=vertcat(*ubg))
else:
    soln = solver(lbx=lbw, ubx=ubw, lbg=vertcat(*lbg), ubg=vertcat(*ubg))


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:    33271
Number of nonzeros in Lagrangian Hessian.............:    11662

Total number of variables............................:      931
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      784
                     variables with only upper bounds:        0
Total number of equa

In [ ]:
X_opt = np.reshape(np.asarray(soln['x'][:n*N]), (N,n)).T
U_opt = np.reshape(np.asarray(soln['x'][n*N:]), (N-1,m)).T


# SCP

In [ ]:
from scp import update_f, update_A, update_B, slerp

In [ ]:
n = 13
m = 6

N = 50

R = np.eye(3) # cost matrix for 2PBVP

# robot parameters
J = np.array([[0.1083, 0.0, 0.0],
            [0.0, 0.1083, 0.0],
            [0.0, 0.0, 0.1083]])
Jxx, Jyy, Jzz = np.diag(J)
Jinv = np.linalg.inv(J)
mass = 7.0

hard_limit_vel   = 0.50
hard_limit_accel = 0.10
hard_limit_omega = 45*np.pi/180
hard_limit_alpha = 50*np.pi/180

hard_limit_accel = 5000.
hard_limit_alpha   = 5000.

# state box constraints
Xlb = np.array([-np.inf,-np.inf,-np.inf, 
    -hard_limit_vel/np.sqrt(3),-hard_limit_vel/np.sqrt(3),-hard_limit_vel/np.sqrt(3),
    -1.0,-1.0,-1.0,0.0,
    -hard_limit_omega/np.sqrt(3),-hard_limit_omega/np.sqrt(3),-hard_limit_omega/np.sqrt(3)])
Xub = np.array([np.inf,np.inf,np.inf, 
    hard_limit_vel/np.sqrt(3),hard_limit_vel/np.sqrt(3),hard_limit_vel/np.sqrt(3),
    1.0,1.0,1.0,1.0,
    hard_limit_omega/np.sqrt(3),hard_limit_omega/np.sqrt(3),hard_limit_omega/np.sqrt(3)])

# control box constraints
Jmin = np.min(np.diag(J))
Ulb = np.array([-mass*hard_limit_accel/np.sqrt(3), -mass*hard_limit_accel/np.sqrt(3), -mass*hard_limit_accel/np.sqrt(3), -Jmin*hard_limit_alpha/np.sqrt(3), -Jmin*hard_limit_alpha/np.sqrt(3), -Jmin*hard_limit_alpha/np.sqrt(3)])
Uub = np.array([mass*hard_limit_accel/np.sqrt(3), mass*hard_limit_accel/np.sqrt(3), mass*hard_limit_accel/np.sqrt(3),  Jmin*hard_limit_alpha/np.sqrt(3), Jmin*hard_limit_alpha/np.sqrt(3), Jmin*hard_limit_alpha/np.sqrt(3)])

# time constraints
taulb = np.array([0.1])
tauub = np.array([100.0])
tau_guess = 50.0 

# param_dict =  {'n':n, 'm':m, 'N':N, 'R':R, 'slew_weighting':slew_weighting, 'max_se3_dist':max_se3_dist, 'J':J, 'Jinv':Jinv, 'mass':mass, 
#       'hard_limit_vel':hard_limit_vel, 'hard_limit_accel':hard_limit_accel, 'hard_limit_omega':hard_limit_omega, 'hard_limit_alpha':hard_limit_alpha, 
#       'Xlb':Xlb, 'Xub':Xub, 'Ulb':Ulb, 'Uub':Uub, 'taulb':taulb, 'tauub':tauub, 'tau_guess':tau_guess}

params = {}
params['mass'] = mass
params['J'] = J
params['hard_limit_vel'] = hard_limit_vel
params['hard_limit_accel'] = hard_limit_accel
params['hard_limit_omega'] = hard_limit_omega
params['hard_limit_alpha'] = hard_limit_alpha

Xi = np.array([3.,3.,3., 0.,0.,0.,1., 0.05,-0.05,0.02, 0.01,-0.01,0.01])

Xref = DM([0.,0.1,0., 0.5,0.5,0.5,0.5, 0.,0.,0., 0.,0.,0.])

In [ ]:
Xprev, Uprev = np.zeros((n,N)), np.zeros((m,N-1))
for ii in range(3):
    Xprev[ii,:] = np.linspace(Xi[ii], np.array(Xref)[ii], num=N).flatten()
    Xprev[7+ii,:] = np.linspace(Xi[7+ii], np.array(Xref)[7+ii], num=N).flatten()
    Xprev[10+ii,:] = np.linspace(Xi[10+ii], np.array(Xref)[10+ii], num=N).flatten()

qi = Xi[3:7].flatten()
qf = np.array(Xref)[3:7].flatten()
for idx, time in enumerate(np.linspace(0.,1., N)):
    Xprev[3:7, idx] = slerp(qi, qf, time)

fs, As, Bs = [], [], []
for idx in range(N-1):
    fs.append(update_f(Xprev[:,idx], Uprev[:,idx], params))
    As.append(update_A(Xprev[:,idx], Uprev[:,idx], params))
    Bs.append(update_B(Xprev[:,idx], Uprev[:,idx], params))

Declare model variables

In [ ]:
rx = MX.sym('rx');    ry = MX.sym('ry');    rz = MX.sym('rz');
qx = MX.sym('qx');    qy = MX.sym('qy');    qz = MX.sym('qz');    qw = MX.sym('qw');
vx = MX.sym('vx');    vy = MX.sym('vy');    vz = MX.sym('vz');
wx = MX.sym('wx');    wy = MX.sym('wy');    wz = MX.sym('wz');
quat = vertcat(qx,qy,qz,qw)
state = vertcat(rx,ry,rz,qx,qy,qz,qw,vx,vy,vz,wx,wy,wz)

Fx = MX.sym('Fx');    Fy = MX.sym('Fy');    Fz = MX.sym('Fz');
Mx = MX.sym('Mx');    My = MX.sym('My');    Mz = MX.sym('Mz');
control = vertcat(Fx,Fy,Fz,Mx,My,Mz)

Start with empty NLP

In [ ]:
w = []    # the decision variables lbw < w < ubw
lbw = []  # lower bound constraint on dec var
ubw = []  # upper bound constraint on dec var
g = []    # vector for constraints lbg < g(w,p) < ubg
lbg = []  # lower bound on constraints
ubg = []  # upper bound on constraints

Declare decision variables

In [ ]:
X = []
for k in range(N):
    Xn = MX.sym('X_'+str(k), n)
    X += [Xn]
    w += [Xn]
    lbw += Xlb.tolist()
    ubw += Xub.tolist()

# control
U = []
for k in range(N-1):
    Un = MX.sym('U_' + str(k), m)
    U += [Un]
    lbw += Ulb.tolist()
    ubw += Uub.tolist()
    w   += [Un]

Discrete update equation

In [ ]:
# #  ODE right hand side function
# f = Function('f', [state,control],[statedot])

# # Integrate with Explicit Euler over 0.2 seconds
# dh = 0.01  # Time step
# xj = state
# n_steps = int(10)
# for ii in range(n_steps):
#     fj = f(xj,control)
#     xj += dh*fj

# # Discrete time dynamics function
# F = Function('F', [state,control],[xj])

Dynamics propagation

In [ ]:
X0 = X[0]
Q = X0
for ii in range(N-1):
    dh = tau_guess / (N-1)
    Ak, Bk = np.eye(n) + dh*As[ii], dh*Bs[ii]
    ck = dh * (fs[ii] - As[ii] @ Xprev[:,ii] - Bs[ii] @ Uprev[:,ii])
    Q = Ak @ X[ii] + Bk @ U[ii] + ck - X[ii+1]
    g += [Q]
    lbg += [np.zeros(n).tolist()]
    ubg += [np.zeros(n).tolist()]

Cost function

In [ ]:
Xi = np.array([3.,3.,3., 0.,0.,0.,1., 0.05,-0.05,0.02, 0.01,-0.01,0.01])

Xref = DM([0.,0.1,0., 0.5,0.5,0.5,0.5, 0.,0.,0., 0.,0.,0.])

J = 0.
for ii in range(N):
    stage_cost = (rx-Xref[0])**2 + (ry-Xref[1])**2 + (rz-Xref[2])**2 + \
        casadi.arccos(casadi.dot(Xref[3:7], quat)) + \
        vx**2 + vy**2 + vz**2 + wx**2 + wy**2 + wz**2 
    stage_cost = casadi.dot(X[ii] - Xref, X[ii] - Xref)
    J += stage_cost

In [ ]:
prob = {'f': J, 'x': vertcat(*w), 'g': vertcat(*g)}
opts = {'verbose':False}

solver = nlpsol('S', 'ipopt', prob);
# solver = qpsol('S', 'osqp', prob)

In [ ]:
def update_dynamics(Xprev, Uprev, params):
    N = Xprev.shape[1]
    fs, As, Bs = [], [], []
    for idx in range(N-1):
        fs.append(update_f(Xprev[:,idx], Uprev[:,idx], params))
        As.append(update_A(Xprev[:,idx], Uprev[:,idx], params))
        Bs.append(update_B(Xprev[:,idx], Uprev[:,idx], params))
    return fs, As, Bs

def normalize_quaternion(Xprev):
    q = Xprev[3:7]
    Xprev[3:7] = q / np.linalg.norm(q, ord=2)
    return Xprev

In [ ]:
max_iter = 5

Xprev, Uprev = np.zeros((n,N)), np.zeros((m,N-1))
for ii in range(3):
    Xprev[ii,:] = np.linspace(Xi[ii], np.array(Xref)[ii], num=N).flatten()
    Xprev[7+ii,:] = np.linspace(Xi[7+ii], np.array(Xref)[7+ii], num=N).flatten()
    Xprev[10+ii,:] = np.linspace(Xi[10+ii], np.array(Xref)[10+ii], num=N).flatten()

qi = Xi[3:7].flatten()
qf = np.array(Xref)[3:7].flatten()
for idx, time in enumerate(np.linspace(0.,1., N)):
    Xprev[3:7, idx] = slerp(qi, qf, time)

fs, As, Bs = update_dynamics(Xprev, Uprev, params)

for itr in range(max_iter):
    use_guess = True
    if use_guess:
        w0 = [] # initial guess of decision variables
        for ii in range(N):
            w0 += Xprev[:,ii].tolist()
        for ii in range(N-1):
            w0 += Uprev[:,ii].tolist()
        soln = solver(x0=w0, lbx=lbw, ubx=ubw, lbg=vertcat(*lbg), ubg=vertcat(*ubg))
    else:
        soln = solver(lbx=lbw, ubx=ubw, lbg=vertcat(*lbg), ubg=vertcat(*ubg))

    if not solver.stats()['success']:
        print('Solver failed at iteration {}'.format(itr))
        break

    Xprev = np.reshape(np.asarray(soln['x'][:n*N]), (N,n)).T
    Uprev = np.reshape(np.asarray(soln['x'][n*N:]), (N-1,m)).T
    for ii in range(N):
        Xprev[:,ii] = normalize_quaternion(Xprev[:,ii])

    fs, As, Bs = update_dynamics(Xprev, Uprev, params)

In [ ]:
solver.stats().keys()

# TAMP Class


In [71]:
from astrobee_tamp import AstrobeeTAMP

Xi = np.array([3.,3.,3., 0.,0.,0.,1., 0.05,-0.05,0.02, 0.01,-0.01,0.01])

Xref = DM([0.,0.1,0., 0.5,0.5,0.5,0.5, 0.,0.,0., 0.,0.,0.])

at = AstrobeeTAMP(Xi, Xref)

at.plan = plan
at.init_straightline()

In [72]:
at.unroll_plan()
at.construct_problem()
at.solve_problem()

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      539
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      452
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

True